In this notebook, I run one step for TwoStreamControlModel to get intermediate outputs which I can then compare to my local run

___

In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # needed to make torch deterministic

In [2]:
import scripts.control_utils as cu
import torch
from PIL import Image
torch.set_printoptions(linewidth=200)

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [3]:
torch.use_deterministic_algorithms(True)

In [4]:
path_to_config = 'ControlNet-XS/configs/inference/sdxl/sdxl_encD_canny_48m.yaml'

If this results in the kernel crashing, I'm using too much GPU memory elsewhere. Shut down every other kernel and try again.

In [5]:
model = cu.create_model(path_to_config).to('cuda')

Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 320, out-chn: 320, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 640, out-chn: 640, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
constructing

In [6]:
size = 768
num_samples=1
prompt='cinematic, shoe in the streets, made from meat, photorealistic shoe, highly detailed'
n_prompt='lowres, bad anatomy, worst quality, low quality'

In [7]:
def get_canny_edges():
    image_path = 'input_images/shoe.png' # chosen to fit size above
    image = cu.get_image(image_path, size=size)
    edges = cu.get_canny_edges(image, low_th=100, high_th=250)
    return edges
edges = get_canny_edges()

Let's play with text encoders. Goal: Get all hidden states for both text encoders.

In [8]:
from util import cls_name

In [9]:
txt_encoder_1 = model.conditioner.embedders[0]
txt_encoder_2 = model.conditioner.embedders[1]
cls_name(txt_encoder_1),cls_name(txt_encoder_2)

('FrozenCLIPEmbedder', 'FrozenOpenCLIPEmbedder2')

In [18]:
def forward_of_FrozenCLIPEmbedder(self, text):
    batch_encoding = self.tokenizer(
        text,
        truncation=True,
        max_length=self.max_length,
        return_length=True,
        return_overflowing_tokens=False,
        padding="max_length",
        return_tensors="pt",
    )
    tokens = batch_encoding["input_ids"].to(self.device)
    outputs = self.transformer(
        input_ids=tokens, output_hidden_states=self.layer == "hidden"
    )
    if self.layer == "last":
        z = outputs.last_hidden_state
    elif self.layer == "pooled":
        z = outputs.pooler_output[:, None, :]
    else:
        z = outputs.hidden_states[self.layer_idx]
    if self.return_pooled:
        return z, outputs.pooler_output
    return z

import open_clip
def forward_of_FrozenOpenCLIPEmbedder2(self, text):
    tokens = open_clip.tokenize(text)
    z = self.encode_with_transformer(tokens.to(self.device))
    if not self.return_pooled and self.legacy:
        return z
    if self.return_pooled:
        assert not self.legacy
        return z[self.layer], z["pooled"]
    return z[self.layer]

In [11]:
encoded_1_txt = txt_encoder_1(prompt)
encoded_1_txt

tensor([[[-3.8918, -2.5114,  4.7184,  ...,  1.1266, -2.2387, -1.2507],
         [ 0.2655, -0.0114,  0.6279,  ..., -0.5496, -0.1617, -0.1203],
         [ 1.0162, -0.1704,  0.6562,  ..., -0.2048,  0.1218,  0.8765],
         ...,
         [ 0.2106, -0.4381, -0.0386,  ..., -1.1807, -0.8254, -0.1423],
         [ 0.2137, -0.4490, -0.0561,  ..., -1.1906, -0.8379, -0.1446],
         [ 0.1943, -0.4328,  0.0195,  ..., -1.2042, -0.8573, -0.1944]]], device='cuda:0')

In [12]:
encoded_1_txt_manual = forward_of_FrozenCLIPEmbedder(txt_encoder_1, prompt)
encoded_1_txt_manual

tensor([[[-3.8917, -2.5113,  4.7167,  ...,  1.1270, -2.2396, -1.2505],
         [ 0.2660, -0.0113,  0.6275,  ..., -0.5494, -0.1619, -0.1202],
         [ 1.0159, -0.1706,  0.6558,  ..., -0.2045,  0.1210,  0.8754],
         ...,
         [ 0.2110, -0.4372, -0.0396,  ..., -1.1811, -0.8259, -0.1430],
         [ 0.2137, -0.4484, -0.0568,  ..., -1.1910, -0.8385, -0.1451],
         [ 0.1941, -0.4325,  0.0185,  ..., -1.2047, -0.8581, -0.1952]]], device='cuda:0')

In [13]:
from sgm.util import autocast

In [14]:
encoded_1_txt_manual_autocasted = autocast(forward_of_FrozenCLIPEmbedder)(txt_encoder_1, prompt)
encoded_1_txt_manual_autocasted

tensor([[[-3.8918, -2.5114,  4.7184,  ...,  1.1266, -2.2387, -1.2507],
         [ 0.2655, -0.0114,  0.6279,  ..., -0.5496, -0.1617, -0.1203],
         [ 1.0162, -0.1704,  0.6562,  ..., -0.2048,  0.1218,  0.8765],
         ...,
         [ 0.2106, -0.4381, -0.0386,  ..., -1.1807, -0.8254, -0.1423],
         [ 0.2137, -0.4490, -0.0561,  ..., -1.1906, -0.8379, -0.1446],
         [ 0.1943, -0.4328,  0.0195,  ..., -1.2042, -0.8573, -0.1944]]], device='cuda:0')

In [15]:
(encoded_1_txt-encoded_1_txt_manual).abs().mean()

tensor(0.0005, device='cuda:0')

In [16]:
(encoded_1_txt-encoded_1_txt_manual_autocasted).abs().mean()

tensor(0., device='cuda:0')

In [23]:
encoded_2_txt = txt_encoder_2(prompt)[0]  # 0 = text conditioning; 1 = text part of time conditioning
encoded_2_txt_manual = forward_of_FrozenOpenCLIPEmbedder2(txt_encoder_2, prompt)[0]
encoded_2_txt_manual_autocasted = autocast(forward_of_FrozenOpenCLIPEmbedder2)(txt_encoder_2, prompt)[0]

In [26]:
(encoded_2_txt-encoded_2_txt_manual).abs().mean(), (encoded_2_txt-encoded_2_txt_manual_autocasted).abs().mean()

(tensor(0.0006, device='cuda:0'), tensor(0., device='cuda:0'))

In [29]:
torch.save(encoded_1_txt_manual, 'intermediate_output/text emb/cloud_1.pt')
torch.save(encoded_2_txt_manual, 'intermediate_output/text emb/cloud_2.pt')